# Open double-funded channel without intermediary

In [1]:
%load_ext autoreload
%autoreload 2

## Setup LND clients for Ali and Bob

In [2]:
%%sh
sudo cp /ali-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/ali.macaroon
sudo chmod +r /tmp/ali.macaroon

sudo cp /bob-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/bob.macaroon
sudo chmod +r /tmp/bob.macaroon

In [3]:
from p2oc.lnd_rpc import LndRpc, lnmsg

In [4]:
ali = LndRpc(host='ali-lnd:10009', cert_path='/ali-lnd/tls.cert', macaroon_path='/tmp/ali.macaroon')
bob = LndRpc(host='bob-lnd:10009', cert_path='/bob-lnd/tls.cert', macaroon_path='/tmp/bob.macaroon')

## Fund Ali and Bob

In [5]:
import os
import time

import bitcoin
from p2oc.btc_rpc import Proxy, Config

In [6]:
bitcoin.SelectParams('regtest')

# Only used to mine initial coins for Ali and Bob
brpc = Proxy(config=Config(
    rpcuser=os.environ['BTCD_RPCUSER'],
    rpcpassword=os.environ['BTCD_RPCPASS'],
    rpcconnect='bitcoind',
    rpcport=18443
))

brpc.getblockcount()

234

In [7]:
# Fund ali and bob
ali_addr = ali.lnd.NewAddress(lnmsg.NewAddressRequest(type=lnmsg.AddressType.WITNESS_PUBKEY_HASH))
_ = list(brpc.generatetoaddress(2, ali_addr.address))

bob_addr = bob.lnd.NewAddress(lnmsg.NewAddressRequest(type=lnmsg.AddressType.WITNESS_PUBKEY_HASH))
_ = list(brpc.generatetoaddress(2, bob_addr.address))

try:
    brpc.createwallet('miner')
except:
    brpc.loadwallet('miner')

miner_addr = brpc.getnewaddress("coinbase")
brpc.unloadwallet('miner')

# Unlock mined coins
_ = list(brpc.generatetoaddress(110, miner_addr))

In [8]:
ali.lnd.WalletBalance(lnmsg.WalletBalanceRequest())

total_balance: 24999892950
confirmed_balance: 24999892950
account_balance {
  key: "default"
  value {
    confirmed_balance: 24999892950
  }
}

In [9]:
bob.lnd.WalletBalance(lnmsg.WalletBalanceRequest())

total_balance: 24983999859
confirmed_balance: 24983999859
account_balance {
  key: "default"
  value {
    confirmed_balance: 24983999859
  }
}

In [10]:
# Give bitcoind a chance to catch up
time.sleep(2)

## Perform double-funded channel creation

In [11]:
import bitcointx.core as bc

import p2oc
from p2oc.offer import deserialize_psbt

In [12]:
# Premium Ali (Taker) is willing to pay
premium_amount = int(0.001 * bc.CoreCoinParams.COIN)

# The requested inbound capacity (from Bob the Maker)
fund_amount = int(0.16 * bc.CoreCoinParams.COIN)

In [13]:
# Ali (Taker) creates offer (to request inbound liquidity in exchange for a fee [the premium])
offer_psbt = p2oc.create_offer(premium_amount, fund_amount, ali)

In [14]:
# Bob (Maker) accepts offer and sends reply
unsigned_psbt = p2oc.accept_offer(offer_psbt, bob)

In [15]:
# Ali checks offer reply and opens pending channel
half_signed_psbt = p2oc.open_channel(unsigned_psbt, ali)

In [16]:
# Bob checks pending channel reply, signs and commits funding tx
p2oc.finalize_offer(half_signed_psbt, bob)

## Confirm channel creation

In [17]:
_ = list(brpc.generatetoaddress(6, miner_addr))

# Give daemons a chance to catch up
time.sleep(2)

In [18]:
# Should be no pending channels
bob.lnd.PendingChannels(lnmsg.PendingChannelsRequest())

In [19]:
bob.lnd.ListChannels(lnmsg.ListChannelsRequest())

channels {
  active: true
  remote_pubkey: "02b26555d2bec2aae0a996528d1491e42147e21eb8e16b00ab33c8cb2867f22abf"
  channel_point: "dddb4b9a9590a2d2f40c4436ede03949d7a1fccf20933e5249d3461793656936:2"
  chan_id: 251788162826242
  capacity: 16100000
  local_balance: 16000000
  remote_balance: 90950
  commit_fee: 9050
  commit_weight: 724
  fee_per_kw: 12500
  csv_delay: 1934
  chan_status_flags: "ChanStatusDefault"
  local_chan_reserve_sat: 161000
  remote_chan_reserve_sat: 161000
  static_remote_key: true
  lifetime: 787
  uptime: 787
  commitment_type: STATIC_REMOTE_KEY
  push_amount_sat: 16000000
  local_constraints {
    csv_delay: 1934
    chan_reserve_sat: 161000
    dust_limit_sat: 573
    max_pending_amt_msat: 15939000000
    min_htlc_msat: 1
    max_accepted_htlcs: 483
  }
  remote_constraints {
    csv_delay: 1934
    chan_reserve_sat: 161000
    dust_limit_sat: 573
    max_pending_amt_msat: 15939000000
    min_htlc_msat: 1
    max_accepted_htlcs: 483
  }
}
channels {
  active: t